In [2]:
# !pip install --upgrade pandas_profiling
!pip install --upgrade category_encoders
# !pip install --upgrade sweetviz

     |████████████████████████████████| 86 kB 3.6 MB/s 


In [3]:
import numpy as np
import pandas as pd
import json
import category_encoders as ce
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
import sklearn
import lightgbm

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor

from sklearn.preprocessing import StandardScaler

#Modeling

##Load dataset

In [5]:
final_cars = pd.read_csv('/content/final_cars.csv')
cars_org = final_cars.copy()

In [6]:
final_cars.sort_values(by='km_driven', ascending=False).head(30)

,brand,engine_capacity,km_driven,model,price,seats,year,manual,semi-automatic,electric,...,imported,coupe,crossover,cuv,hatchback,mpv,pickup,sedan,suv,van
20848,toyota,1.30,4.294967e+09,toyota yaris,235000000,5,2008.0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
19329,toyota,1.50,2.100000e+08,toyota vios,484000000,4,2018.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
10367,kia,1.10,1.520000e+08,kia morning,145000000,5,2010.0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3052,ford,2.40,1.200000e+08,ford transit,395000000,4,2017.0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
7623,hyundai,2.00,1.000000e+08,hyundai genesis,430000000,2,2009.0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
22739,toyota,1.60,9.500000e+07,toyota corolla altis,360000000,5,2009.0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
25213,mazda,2.20,7.200000e+07,mazda cx5,630000000,5,2016.0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
9307,kia,1.60,7.000000e+07,kia k3,412000000,4,2015.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9664,kia,2.20,7.000000e+07,kia sedona,920000000,5,2018.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
34244,ford,2.50,5.300000e+07,ford everest,748000000,7,2013.0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


##Remove outliers

In [7]:
final_cars.drop(index=[32267, 43234], inplace=True) #Contains wrong information

In [8]:
final_cars.drop(index=final_cars[final_cars['seats'] == 'Khác'].index, inplace=True)
final_cars['seats'] = final_cars.seats.astype(float)

In [9]:
final_cars = final_cars[(np.abs(stats.zscore(final_cars.price)) < 2.8)]

In [10]:
replace_cc = final_cars[final_cars.engine_capacity == 2000].index
final_cars.engine_capacity.loc[replace_cc] = 2.0

In [11]:
final_cars = final_cars[final_cars.seats<30]
final_cars = final_cars[final_cars.year>2007]
final_cars = final_cars[final_cars.km_driven < 1e+08]
final_cars = final_cars[final_cars.km_driven > 0]

In [12]:
index_drop = []
list_drop = [NameError]
for i in final_cars.model.unique():
  idx = final_cars[final_cars.model == i][(np.abs(stats.zscore(final_cars[final_cars.model == i].price)) > 2.75)].index.tolist()
  index_drop.append(idx)

for i in index_drop:
  for k in i:
    list_drop.append(k)

list_drop = list_drop[1:]
len(list_drop)

195

In [13]:
final_cars.drop(index = list_drop, inplace=True)

In [14]:
final_cars.sort_values(by='price', ascending = False)

,brand,engine_capacity,km_driven,model,price,seats,year,manual,semi-automatic,electric,...,imported,coupe,crossover,cuv,hatchback,mpv,pickup,sedan,suv,van
39766,mercedes,2.0,3500.0,mercedes glc,1759000000,5.0,2021.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
13654,mercedes,2.0,24000.0,mercedes e,1759000000,4.0,2017.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
27346,mercedes,2.0,43452.0,mercedes glc,1759000000,5.0,2018.0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
44244,mercedes,2.0,24000.0,mercedes e,1759000000,5.0,2017.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
39862,mercedes,2.0,3000.0,mercedes glc,1759000000,5.0,2020.0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,chevrolet,0.8,70000.0,chevrolet spark,102000000,5.0,2012.0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
42860,chevrolet,0.9,70000.0,chevrolet spark,102000000,2.0,2012.0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
40422,daewoo,0.8,90.0,daewoo matiz,100000000,5.0,2008.0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
44378,daewoo,0.8,10000.0,daewoo matiz,99900000,5.0,2008.0,1,0,0,...,1,0,0,0,1,0,0,0,0,0


##Encoding categorical

In [15]:
final_cars['brand'] = ce.JamesSteinEncoder().fit_transform(final_cars['brand'], final_cars['price'])
final_cars['model'] = ce.JamesSteinEncoder().fit_transform(final_cars['model'], final_cars['price'])

##Train test split

In [16]:
X = final_cars.drop(["price"],axis=1)
y = final_cars['price']

In [17]:
VERBOSE = 0
SEED = 42

In [27]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

X_train , X_test, y_train, y_test = train_test_split(X, y, random_state = SEED , test_size=0.2)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [30]:
regressors = [
    LinearRegression(),
    # SGDRegressor(),
    # ElasticNet(),
    # KernelRidge(),
    # SVR(),
    XGBRegressor(),
    RandomForestRegressor(),
    ExtraTreesRegressor(), 
    GradientBoostingRegressor()
]

In [31]:
res = pd.DataFrame(columns=['Model', 'RMSE', 'R-Squared'])
RMSE = []
R2 = []
for num, regressor in enumerate(regressors):
    reg = regressor.fit(X_train, y_train)
    pred = reg.predict(X_test)
    rmse = np.sqrt(mean_squared_error(pred, y_test))
    r2 = r2_score(pred, y_test)
    RMSE.append(str(np.around(rmse)))
    R2.append(str(round(r2, 2)*100) + '%')
res['Model'] = ['LinearRegression', 'XGBRegressor', 'RandomForestRegressor', 'ExtraTreesRegressor', 'GradientBoostingRegressor']
res['RMSE'] = RMSE
res['R-Squared'] = R2
res

[19:52:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


,Model,RMSE,R-Squared
0,LinearRegression,147970449.0,69.0%
1,XGBRegressor,96498420.0,88.0%
2,RandomForestRegressor,67328653.0,95.0%
3,ExtraTreesRegressor,71713363.0,94.0%
4,GradientBoostingRegressor,97396914.0,88.0%


###GridSearchCV

In [22]:
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# param_grid2 = {
#     "learning_rate": [0.2, 0.25],
#     "n_estimators": [i for i in range(270,300,10)],
#     'max_depth': [i for i in range(13,19)],
#     "num_leaves": [i for i in range(30,50)],
#     'min_data_in_leaf': [20,23,24,19],
#     'reg_alpha': [0.1, 0.2],
#     'reg_lambda': [0.02,0.01]
# }

# regressor = LGBMRegressor()
# gscv_lgbm2 = GridSearchCV(regressor, param_grid2, scoring='neg_mean_squared_error', cv=3)
# gscv_lgbm2.fit(X_train, y_train)
# yhat_lgbmr = gscv_lgbm2.predict(X_test)
# print(gscv_lgbm2.best_params_)
# np.sqrt(mean_squared_error(y_test, yhat_lgbmr))

# # for x in range(0,50):
# #   gscv_lgbm2 = RandomizedSearchCV(regressor, param_grid2, scoring='neg_mean_squared_error', cv=3)
# #   gscv_lgbm2.fit(X_train, y_train)
# #   yhat_lgbmr = gscv_lgbm2.predict(X_test)
# #   k = np.sqrt(mean_squared_error(y_test, yhat_lgbmr))
# #   if k < 67596171:
# #     print(gscv_lgbm2.best_params_)
# #     print(k)

66.0 : {'reg_lambda': 0.01, 'reg_alpha': 0.2, 'num_leaves': 40, 'n_estimators': 300, 'min_data_in_leaf': 19, 'max_depth': 13, 'learning_rate': 0.2}

66.1 : {'reg_lambda': 0.01, 'reg_alpha': 0.1, 'num_leaves': 47, 'n_estimators': 310, 'min_data_in_leaf': 20, 'max_depth': 18, 'learning_rate': 0.2}

65.4 : {'reg_lambda': 0.01, 'reg_alpha': 0.4, 'num_leaves': 39, 'n_estimators': 245, 'min_data_in_leaf': 20, 'max_depth': 13, 'learning_rate': 0.25}

65.2: {'reg_lambda': 0.02, 'reg_alpha': 0.2, 'num_leaves': 48, 'n_estimators': 280, 'min_data_in_leaf': 19, 'max_depth': 14, 'learning_rate': 0.2}

In [23]:
regressor = LGBMRegressor()
params = {'reg_lambda': 0.02, 'reg_alpha': 0.2, 'num_leaves': 48, 'n_estimators': 280, 'min_data_in_leaf': 19, 'max_depth': 14, 'learning_rate': 0.2}
# params = {'reg_lambda': 0.01, 'reg_alpha': 0.1, 'num_leaves': 47, 'n_estimators': 310, 'min_data_in_leaf': 20, 'max_depth': 18, 'learning_rate': 0.2}
regressor.set_params(**params)
reg = regressor.fit(X_train, y_train)
pred = reg.predict(X_test)
pred = np.around(pred, decimals=-6) #lam tron gia tien
rmse = np.sqrt(mean_squared_error(pred, y_test))
print(f'{regressor} has rmse of {rmse}')

LGBMRegressor(learning_rate=0.2, max_depth=14, min_data_in_leaf=19,
              n_estimators=280, num_leaves=48, reg_alpha=0.2, reg_lambda=0.02) has rmse of 65215604.48804556


In [24]:
from sklearn.metrics import r2_score

r2_score(pred, y_test)

0.9514788290445542

##Feature Importances plot

In [25]:
def plotImp(model, X , num = 20, fig_size = (40, 20)):
    feature_imp = pd.DataFrame({'Value':model.feature_importances_,'Feature':X.columns})
    plt.figure(figsize=fig_size)
    sns.set(font_scale = 5)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", 
                                                        ascending=False)[0:num])
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances-01.png')
    plt.show()

In [28]:
import plotly.express as px

feature_imp = pd.DataFrame({'Value':reg.feature_importances_,'Feature':X_train.columns})
dat = feature_imp.sort_values(by='Value',ascending=True)
fig = px.bar(dat, x="Value", y="Feature", title="Feature Importances")
fig.show()